<a href="https://colab.research.google.com/github/Alifarki/ViT-pytorch/blob/main/Transformer_Vit_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jul  9 12:23:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/Alifarki/ViT-pytorch.git

Cloning into 'ViT-pytorch'...
remote: Enumerating objects: 170, done.
remote: Total 170 (delta 0), reused 0 (delta 0), pack-reused 170
Receiving objects: 100% (170/170), 21.20 MiB | 32.26 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [3]:
cd ViT-pytorch/

/content/ViT-pytorch


In [4]:
mkdir checkpoint

In [5]:
cd checkpoint/

/content/ViT-pytorch/checkpoint


In [6]:
!wget https://storage.googleapis.com/vit_models/imagenet21k%2Bimagenet2012/ViT-B_16.npz

--2022-07-09 12:24:08--  https://storage.googleapis.com/vit_models/imagenet21k%2Bimagenet2012/ViT-B_16.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347502902 (331M) [application/octet-stream]
Saving to: ‘ViT-B_16.npz’

ViT-B_16.npz        100%[===================>] 331.40M  53.7MB/s    in 6.2s    

2022-07-09 12:24:15 (53.7 MB/s) - ‘ViT-B_16.npz’ saved [347502902/347502902]



In [9]:
cd /content/ViT-pytorch

/content/ViT-pytorch


In [10]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 9901, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 9901 (delta 4), reused 6 (delta 0), pack-reused 9881
Receiving objects: 100% (9901/9901), 14.85 MiB | 30.11 MiB/s, done.
Resolving deltas: 100% (6809/6809), done.


In [11]:
cd apex

/content/ViT-pytorch/apex


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir ./

In [13]:
cd ..

/content/ViT-pytorch


In [14]:
!pip install ml_collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 1.7 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=12f2148359882b5d1f99036b9c41ee6208a93aabeed68838c95ed6f819e0a236
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
cp /content/drive/MyDrive/csc532.zip /content/ViT-pytorch/Data-cov

In [22]:
cd Data-cov/

/content/ViT-pytorch/Data-cov


In [ ]:
!unzip csc532.zip

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet34
import os

In [ ]:
# import os
# from shutil import copyfile
# from scipy.io import loadmat

# images_path = "./JPEGImages"
# labels_path = "./ImageSplits"
# new_dataset_path = "./StanfordActionDataset"

# if not(os.path.exists(new_dataset_path)):
#     os.mkdir(new_dataset_path)
#     os.mkdir(new_dataset_path + '/' + 'train')
#     os.mkdir(new_dataset_path + '/' + 'test')

# txts = os.listdir(labels_path)
# for txt in txts:
#     idx = txt[0:-4].rfind('_')
#     class_name = txt[0:idx]
#     if class_name in ['actions.tx', 'test.tx', 'train.tx']:
#       continue
#     train_or_test = txt[idx+1:-4]
#     txt_contents = open(labels_path + '/' + txt)
#     txt_contents = txt_contents.read()
#     image_names  = txt_contents.split('\n')
#     num_aid_images_per_class = 1
#     for image_name in image_names[0:-1]:
#         if not(os.path.exists(new_dataset_path + '/' + train_or_test + '/' + class_name)):
#             os.mkdir(new_dataset_path + '/' + train_or_test + '/' + class_name)
#         copyfile(images_path + '/' + image_name,
#                  new_dataset_path + '/' + train_or_test + '/' + class_name + '/' + image_name)
        

In [ ]:
# #Data
# transform_train = transforms.Compose([
# transforms.Resize((224, 224)),
# transforms.ToTensor(),
# ])
# transform_test = transforms.Compose([
# transforms.Resize((224, 224)),
# transforms.ToTensor(),
# ])
# trainset = datasets.ImageFolder(root='/content/ViT-pytorch/Data-cov/datasets/datasets/train/',
# transform=transform_train)
# trainloader = torch.utils.data.DataLoader(
# trainset, batch_size=16, shuffle=True, num_workers=2)
# # testset = torchvision.datasets.CIFAR100(
# # root='./data', train=False, download=True, transform=transform_test)
# testset = datasets.ImageFolder(root='/content/ViT-pytorch/Data-cov/datasets/datasets/test/',
# transform=transform_test)
# testloader = torch.utils.data.DataLoader(
# testset, batch_size=16, shuffle=True, num_workers=2)


In [ ]:
# !python train.py --name stanford40-1 --dataset stanford40 --model_type ViT-B_16 --pretrained_dir checkpoint/ViT-B_16.npz --fp16 --fp16_opt_level O2 --gradient_accumulation_steps 16

In [25]:
cd /content/ViT-pytorch

/content/ViT-pytorch


In [26]:
!python train.py --name cov19 --dataset cov19 --model_type ViT-B_16  --fp16 --fp16_opt_level O2 --gradient_accumulation_steps 16

07/09/2022 12:29:51 - WARNING - __main__ - Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
07/09/2022 12:29:52 - INFO - models.modeling - load_pretrained: resized variant: torch.Size([1, 577, 768]) to torch.Size([1, 197, 768])
load_pretrained: grid-size from 24 to 14
07/09/2022 12:30:04 - INFO - __main__ - classifier: token
hidden_size: 768
patches:
  size: !!python/tuple [16, 16]
representation_size: null
transformer: {attention_dropout_rate: 0.0, dropout_rate: 0.1, mlp_dim: 3072, num_heads: 12,
  num_layers: 12}

07/09/2022 12:30:04 - INFO - __main__ - Training parameters Namespace(dataset='cov19', decay_type='cosine', device=device(type='cuda'), eval_batch_size=64, eval_every=100, fp16=True, fp16_opt_level='O2', gradient_accumulation_steps=16, img_size=224, learning_rate=0.03, local_rank=-1, loss_scale=0, max_grad_norm=1.0, model_type='ViT-B_16', n_gpu=1, name='cov19', num_steps=10000, output_dir='output', pretrained_dir='checkpoint/ViT-

In [ ]:
# batch = next(iter(testloader))
# batch[0].shape, batch[1].shape

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# def normalize_image(image):
#     image_min = image.min()
#     image_max = image.max()
#     image.clamp_(min = image_min, max = image_max)
#     image.add_(-image_min).div_(image_max - image_min + 1e-5)
#     return image

# def plot_images(images, labels, classes, normalize=True):

#     n_images = len(images)

#     rows = int(np.sqrt(n_images))
#     cols = int(np.sqrt(n_images))

#     fig = plt.figure(figsize=(20, 20))

#     for i in range(rows*cols):

#         ax = fig.add_subplot(rows, cols, i+1)
        
#         image = images[i]

#         if normalize:
#             image = normalize_image(image)

#         ax.imshow(image.permute(1, 2, 0).cpu().numpy())
#         ax.set_title(classes[labels[i]])
#         ax.axis('off')

In [ ]:
# classes = testset.classes

# plot_images(batch[0], batch[1], classes, normalize=True)

In [27]:
pwd

'/content/ViT-pytorch'

In [51]:
!git pull origin main

From https://github.com/Alifarki/ViT-pytorch
 * branch            main       -> FETCH_HEAD
Already up to date.


In [71]:
!git config --global user.email "alifarki@yahoo.com"

In [72]:
!git add *.py

In [73]:
!git commit -m "add cov19 Actions"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
	modified:   utils/data_utils.py

Untracked files:
	Data-cov/
	apex/
	checkpoint/
	logs/
	output/

no changes added to commit


In [86]:
!git push -u origin main

fatal: not a git repository (or any of the parent directories): .git
